In [ ]:
import logging
import importlib
importlib.reload(logging) # see https://stackoverflow.com/a/21475297/1469195
log = logging.getLogger()
log.setLevel('INFO')
import sys

logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)

In [ ]:
%%capture
import os
import site
os.sys.path.insert(0, '/home/schirrmr/code/reversible/')
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')
os.sys.path.insert(0, '/home/schirrmr/code/explaining/reversible//')


%load_ext autoreload
%autoreload 2
import numpy as np
import logging
log = logging.getLogger()
log.setLevel('INFO')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'png'
matplotlib.rcParams['figure.figsize'] = (12.0, 1.0)
matplotlib.rcParams['font.size'] = 14
import seaborn
seaborn.set_style('darkgrid')

from reversible2.sliced import sliced_from_samples
from numpy.random import RandomState

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import copy
import math

import itertools
import torch as th
from braindecode.torch_ext.util import np_to_var, var_to_np
from reversible2.splitter import SubsampleSplitter

from reversible2.view_as import ViewAs

from reversible2.affine import AdditiveBlock
from reversible2.plot import display_text, display_close
from reversible2.high_gamma import load_file, create_inputs
from reversible2.high_gamma import load_train_test
th.backends.cudnn.benchmark = True
from reversible2.models import deep_invertible


In [ ]:
rng = RandomState(201904113)#2 ganz gut # 13 sehr gut)

X = rng.randn(5,2) * np.array([1,0])[None] + np.array([-1,0])[None]
X = np.sort(X, axis=0)

X_test = rng.randn(500,2) * np.array([1,0])[None] + np.array([-1,0])[None]
X_test = np.sort(X_test, axis=0)
train_inputs = np_to_var(X[::2], dtype=np.float32)
val_inputs = np_to_var(X[1::2], dtype=np.float32)
test_inputs = np_to_var(X_test, dtype=np.float32)

plt.figure(figsize=(5,5))
plt.scatter(var_to_np(train_inputs)[:,0], var_to_np(train_inputs)[:,1])
plt.scatter(var_to_np(val_inputs)[:,0], var_to_np(val_inputs)[:,1])
plt.scatter([-1],[0], color='black')
plt.xlim(-2.5,5.5)
plt.ylim(-4,4)


In [ ]:
cuda = False
from reversible2.distribution import TwoClassIndependentDist

from reversible2.blocks import dense_add_block
from reversible2.rfft import RFFT, Interleave
from reversible2.util import set_random_seeds
from torch.nn import ConstantPad2d
import torch as th
from reversible2.splitter import SubsampleSplitter
from matplotlib.patches import Ellipse
from reversible2.gaussian import get_gauss_samples


set_random_seeds(2019011641, cuda)
model = nn.Sequential(
    dense_add_block(2,200),
    dense_add_block(2,200),
    dense_add_block(2,200),
    dense_add_block(2,200),
)


dist = TwoClassIndependentDist(2, truncate_to=None)
tr_log_stds = (th.zeros_like(train_inputs, requires_grad=True) - 20).detach().clone().requires_grad_(True)

from reversible2.dist_model import ModelAndDist

model_and_dist = ModelAndDist(model, dist)
optim = th.optim.Adam([{'params': model_and_dist.dist.parameters(), 'lr':1e-2},
                      {'params': list(model_and_dist.model.parameters()),
                      'lr': 1e-3}])
optim_stds = th.optim.Adam([{'params': tr_log_stds, 'lr':1e-2},])

In [ ]:
n_epochs = 10001

for i_epoch in range(n_epochs):
    outs = model_and_dist.model(train_inputs + (th.rand_like(train_inputs) -0.5) * 1e-2)
    outs = outs + (th.randn_like(outs) * th.exp(tr_log_stds))
    nll = -th.mean(model_and_dist.dist.get_total_log_prob(0, outs))
    optim.zero_grad()
    nll.backward()
    optim.step()
    if i_epoch % (n_epochs // 20) == 0:
        tr_out = model_and_dist.model(train_inputs)
        va_out = model_and_dist.model(val_inputs)
        te_out = model_and_dist.model(test_inputs)

        fig = plt.figure(figsize=(5,5))
        mean, std = model_and_dist.dist.get_mean_std(0)
        plt.plot(var_to_np(te_out)[:,0], var_to_np(te_out)[:,1], color=seaborn.color_palette()[1])

        plt.scatter(var_to_np(tr_out)[:,0], var_to_np(tr_out)[:,1], color=seaborn.color_palette()[0])
        plt.scatter(var_to_np(va_out)[:,0], var_to_np(va_out)[:,1], color=seaborn.color_palette()[2])
        ellipse = Ellipse(var_to_np(mean), var_to_np(std[0]), var_to_np(std[1]), edgecolor='black', facecolor='None')
        ax = plt.gca()
        ax.add_artist(ellipse)
        plt.title("Epoch {:d} of {:d}\nTrain NLL {:.1E}\nValid NLL {:.1E}\nTest NLL {:.1E}".format(
            i_epoch, n_epochs,
            -th.mean(model_and_dist.get_total_log_prob(0, train_inputs)).item(),
            -th.mean(model_and_dist.get_total_log_prob(0, val_inputs)).item(),
            -th.mean(model_and_dist.get_total_log_prob(0, test_inputs)).item(),
        ))
        plt.axis('equal') #before not done
        display_close(fig)
        
        examples = model_and_dist.get_examples(0,200)
        fig = plt.figure(figsize=(5,5))
        plt.plot(var_to_np(test_inputs)[:,0], var_to_np(test_inputs)[:,1], color=seaborn.color_palette()[2])
        plt.scatter(var_to_np(examples)[:,0], var_to_np(examples)[:,1], color=seaborn.color_palette()[1])
        plt.scatter(var_to_np(train_inputs)[:,0], var_to_np(train_inputs)[:,1], color=seaborn.color_palette()[0])

        plt.axis('equal')
        plt.title("Input space")
        plt.legend(("Test", "Fake","Train", ),)
        display_close(fig)

    

In [ ]:
tr_out = model_and_dist.model(train_inputs)
va_out = model_and_dist.model(val_inputs)
te_out = model_and_dist.model(test_inputs)

fig = plt.figure(figsize=(5,5))
mean, std = model_and_dist.dist.get_mean_std(0)
plt.plot(var_to_np(te_out)[:,0], var_to_np(te_out)[:,1], color=seaborn.color_palette()[1])

plt.scatter(var_to_np(tr_out)[:,0], var_to_np(tr_out)[:,1], color=seaborn.color_palette()[0])
plt.scatter(var_to_np(va_out)[:,0], var_to_np(va_out)[:,1], color=seaborn.color_palette()[2])
ellipse = Ellipse(var_to_np(mean), var_to_np(std[0]), var_to_np(std[1]), edgecolor='black', facecolor='None')
ax = plt.gca()
ax.add_artist(ellipse)
plt.title("Epoch {:d} of {:d}\nTrain NLL {:.1E}\nValid NLL {:.1E}\nTest NLL {:.1E}".format(
    i_epoch, n_epochs,
    -th.mean(model_and_dist.get_total_log_prob(0, train_inputs)).item(),
    -th.mean(model_and_dist.get_total_log_prob(0, val_inputs)).item(),
    -th.mean(model_and_dist.get_total_log_prob(0, test_inputs)).item(),
))
plt.axis('equal') #before not done

x_start = -1
x_stop = 0.5
y_start = -0.75
y_stop = 1
points = th.stack(th.meshgrid(th.linspace(x_start,x_stop,30),
                              th.linspace(y_start,y_stop,30)), dim=-1)
probs = th.exp(model_and_dist.dist.get_total_log_prob(0,points.view(-1, points.shape[-1]))).view(
    points.shape[:2])
plt.imshow(var_to_np(probs).T, origin='lower left',
                   extent=((x_start, x_stop, y_start, y_stop)), cmap=cm.Reds,
                  aspect='auto', interpolation='bilinear', vmin=0)
display_close(fig)


In [ ]:

examples = model_and_dist.get_examples(0,200)
fig = plt.figure(figsize=(5,5))
plt.plot(var_to_np(test_inputs)[:,0], var_to_np(test_inputs)[:,1], color=seaborn.color_palette()[2])
plt.scatter(var_to_np(examples)[:,0], var_to_np(examples)[:,1], color=seaborn.color_palette()[1], alpha=0.1)
plt.scatter(var_to_np(train_inputs)[:,0], var_to_np(train_inputs)[:,1], color=seaborn.color_palette()[0])

plt.axis('equal')
plt.title("Input space")
plt.legend(("Test", "Fake","Train", ),)

x_start = -3
x_stop = 2
y_start = -1
y_stop = 1
points = th.stack(th.meshgrid(th.linspace(x_start,x_stop,30),
                              th.linspace(y_start,y_stop,30)), dim=-1)
probs = th.exp(model_and_dist.get_total_log_prob(0,points.view(-1, points.shape[-1]))).view(
    points.shape[:2])

#plt.imshow(var_to_np(probs).T, origin='lower left',
#                   extent=((x_start, x_stop, y_start, y_stop)), cmap=cm.Reds,
#                  aspect='auto', interpolation='bilinear', vmin=0, vmax=1)


for std_dev in [1,3]:
    radians = np.linspace(0,2*np.pi,24)
    circle_points = np.stack([np.cos(radians), np.sin(radians)], axis=-1)
    circle_th = np_to_var(circle_points, device=train_inputs.device, dtype=np.float32)
    out_circle = (circle_th * dist.get_mean_std(0)[1] * std_dev) + dist.get_mean_std(0)[0]
    from reversible2.invert import  invert
    in_circle = invert(model_and_dist.model, out_circle)


    plt.plot(var_to_np(in_circle)[:,0], var_to_np(in_circle)[:,1],color=seaborn.color_palette()[3],
                alpha=1, lw=1,)
display_close(fig)

In [ ]:
th.max(probs)

In [ ]:
outs = model_and_dist.model(train_inputs)
print("outs", var_to_np(outs))
demeaned = outs - model_and_dist.dist.get_mean_std(0)[0].unsqueeze(0)
print("demeaned", demeaned)
print("squares", th.sum(demeaned **2, dim=1))
print("squaresmean", th.mean(th.sum(demeaned **2, dim=1)))

In [ ]:
cuda = False
from reversible2.distribution import TwoClassIndependentDist

from reversible2.blocks import dense_add_block
from reversible2.rfft import RFFT, Interleave
from reversible2.util import set_random_seeds
from torch.nn import ConstantPad2d
import torch as th
from reversible2.splitter import SubsampleSplitter
from matplotlib.patches import Ellipse
from reversible2.gaussian import get_gauss_samples


set_random_seeds(2019011641, cuda)
model = nn.Sequential(
    dense_add_block(2,200),
    dense_add_block(2,200),
    dense_add_block(2,200),
    dense_add_block(2,200),
)


dist = TwoClassIndependentDist(2, truncate_to=None)
tr_log_stds = (th.zeros_like(train_inputs, requires_grad=True) + 0).detach().clone().requires_grad_(True)

from reversible2.dist_model import ModelAndDist

model_and_dist = ModelAndDist(model, dist)
optim = th.optim.Adam([{'params': model_and_dist.dist.parameters(), 'lr':1e-2},
                      {'params': list(model_and_dist.model.parameters()),
                      'lr': 1e-3}])
optim_stds = th.optim.Adam([{'params': tr_log_stds, 'lr':1e-2},])

In [ ]:

n_epochs = 10001

for i_epoch in range(n_epochs):
    outs = model_and_dist.model(train_inputs + (th.rand_like(train_inputs) -0.5) * 1e-2)
    outs = outs + (th.randn_like(outs) * th.exp(tr_log_stds))
    nll = -th.mean(model_and_dist.dist.get_total_log_prob(0, outs))
    optim.zero_grad()
    nll.backward()
    optim.step()
    if i_epoch % (n_epochs // 20) == 0:
        tr_out = model_and_dist.model(train_inputs)
        va_out = model_and_dist.model(val_inputs)
        te_out = model_and_dist.model(test_inputs)

        fig = plt.figure(figsize=(5,5))
        mean, std = model_and_dist.dist.get_mean_std(0)
        plt.plot(var_to_np(te_out)[:,0], var_to_np(te_out)[:,1], color=seaborn.color_palette()[1])

        plt.scatter(var_to_np(tr_out)[:,0], var_to_np(tr_out)[:,1], color=seaborn.color_palette()[0])
        plt.scatter(var_to_np(va_out)[:,0], var_to_np(va_out)[:,1], color=seaborn.color_palette()[2])
        ellipse = Ellipse(var_to_np(mean), var_to_np(std[0]), var_to_np(std[1]), edgecolor='black', facecolor='None')
        ax = plt.gca()
        ax.add_artist(ellipse)
        plt.title("Epoch {:d} of {:d}\nTrain NLL {:.1E}\nValid NLL {:.1E}\nTest NLL {:.1E}".format(
            i_epoch, n_epochs,
            -th.mean(model_and_dist.get_total_log_prob(0, train_inputs)).item(),
            -th.mean(model_and_dist.get_total_log_prob(0, val_inputs)).item(),
            -th.mean(model_and_dist.get_total_log_prob(0, test_inputs)).item(),
        ))
        plt.axis('equal') #before not done
        display_close(fig)
        
        examples = model_and_dist.get_examples(0,200)
        fig = plt.figure(figsize=(5,5))
        plt.plot(var_to_np(test_inputs)[:,0], var_to_np(test_inputs)[:,1], color=seaborn.color_palette()[2])
        plt.scatter(var_to_np(examples)[:,0], var_to_np(examples)[:,1], color=seaborn.color_palette()[1])
        plt.scatter(var_to_np(train_inputs)[:,0], var_to_np(train_inputs)[:,1], color=seaborn.color_palette()[0])

        plt.axis('equal')
        plt.title("Input space")
        plt.legend(("Test", "Fake","Train", ),)
        display_close(fig)

    

In [ ]:
outs = model_and_dist.model(train_inputs)
print("outs", var_to_np(outs))
demeaned = outs - model_and_dist.dist.get_mean_std(0)[0].unsqueeze(0)
print("demeaned", demeaned)
print("squares", th.sum(demeaned **2, dim=1))
print("squaresmean", th.mean(th.sum(demeaned **2, dim=1)))

In [ ]:

tr_out = model_and_dist.model(train_inputs)
va_out = model_and_dist.model(val_inputs)
te_out = model_and_dist.model(test_inputs)

fig = plt.figure(figsize=(5,5))
mean, std = model_and_dist.dist.get_mean_std(0)
plt.plot(var_to_np(te_out)[:,0], var_to_np(te_out)[:,1], color=seaborn.color_palette()[1])

plt.scatter(var_to_np(tr_out)[:,0], var_to_np(tr_out)[:,1], color=seaborn.color_palette()[0])
plt.scatter(var_to_np(va_out)[:,0], var_to_np(va_out)[:,1], color=seaborn.color_palette()[2])
ellipse = Ellipse(var_to_np(mean), var_to_np(std[0]), var_to_np(std[1]), edgecolor='black', facecolor='None')
ax = plt.gca()
ax.add_artist(ellipse)
plt.title("Epoch {:d} of {:d}\nTrain NLL {:.1E}\nValid NLL {:.1E}\nTest NLL {:.1E}".format(
    i_epoch, n_epochs,
    -th.mean(model_and_dist.get_total_log_prob(0, train_inputs)).item(),
    -th.mean(model_and_dist.get_total_log_prob(0, val_inputs)).item(),
    -th.mean(model_and_dist.get_total_log_prob(0, test_inputs)).item(),
))
plt.xlim(-0.5,0.7)
plt.ylim(-1,0.2)
display_close(fig)
